# First run the geolife.py script to generate trajectories

In [2]:
#pragma cling add_include_path("/usr/src/movetk/tutorials")

In [3]:
#pragma cling add_include_path("/usr/local/include")

In [4]:
#include "movetk/utils/Geolife.h"

In file included from input_line_15:1:
In file included from /usr/local/include/movetk/utils/Geolife.h:21:
In file included from /usr/local/include/movetk/io/ProbeReader.h:40:
In file included from /miniconda/include/boost/iostreams/copy.hpp:26:
/miniconda/include/boost/bind.hpp:36:1: warning: The practice of declaring the Bind placeholders (_1, _2, ...) in the global namespace is deprecated. Please use
      &lt;boost/bind/bind.hpp&gt; + using namespace boost::placeholders, or define BOOST_BIND_GLOBAL_PLACEHOLDERS to retain the current
      behavior. [-W#pragma-messages]
BOOST_PRAGMA_MESSAGE(
^
/miniconda/include/boost/config/pragma_message.hpp:24:34: note: expanded from macro &#39;BOOST_PRAGMA_MESSAGE&#39;
# define BOOST_PRAGMA_MESSAGE(x) _Pragma(BOOST_STRINGIZE(message(x)))
                                 ^
&lt;scratch space&gt;:7:2: note: expanded from here
 message(&quot;The practice of declaring the Bind placeholders (_1, _2, ...) &quot; &quot;in the global namespace is depreca

In [5]:
?ProbeReader

http://127.0.0.1/movetk.reference/class_probe_reader.html

In [6]:
// Create trajectory reader
std::unique_ptr<ProbeReader<ProbeTraits>> probe_reader;

In [5]:
std::string fName = "/usr/src/movetk/tutorials/out/117.csv";

In [6]:
probe_reader = ProbeReaderFactory::create<ProbeTraits>(fName.c_str());

In [7]:
?TrajectoryReader

http://192.168.99.100/movetk.reference/class_trajectory_reader.html

In [8]:
//type of probe iterator
using ProbeInputIterator = decltype(probe_reader->begin());

In [9]:
auto trajectory_reader = TrajectoryReader<TrajectoryTraits, ProbeInputIterator>(probe_reader->begin(), probe_reader->end());
 // Create an output geojson file

In [10]:
// std::ofstream ofjson("output_trajectories_010.geojson");
// ofjson.setf(std::ios::fixed);

In [11]:
std::vector<TrajectoryTraits::trajectory_type> trajectories;
for (auto t: trajectory_reader){
    trajectories.push_back(t);
}

In [12]:
std::cout<<trajectories.size()<<"\n";

3


In [13]:
movetk_io::trajectories_to_geojson<typename std::vector<typename TrajectoryTraits::trajectory_type>::iterator,
                                      ProbeTraits::ProbeColumns::LAT,
                                      ProbeTraits::ProbeColumns::LON>(std::cout,
                                                                      trajectories.begin(),
                                                                      std::prev(trajectories.end()));

{
    "features": [
{
            "geometry": {
                "coordinates": [
[116.333383, 40.00156667],
[116.33356667, 39.99948333],
[116.33260000, 39.99851667],
[116.33176667, 39.99695000],
[116.33155000, 39.99213333],
[116.33140000, 39.99183333],
[116.33180000, 39.98990000],
[116.33183333, 39.98856667],
[116.33175000, 39.98746667],
[116.33188333, 39.98563333],
[116.33290000, 39.97956667],
[116.33363333, 39.97568333],
[116.33365000, 39.97530000],
[116.33323333, 39.97538333],
[116.33366667, 39.97536667],
[116.33286667, 39.97540000],
[116.33203333, 39.97543333],
[116.33173333, 39.97555000],
[116.33155000, 39.97575000],
[116.33126667, 39.97576667]
               ],
                "type": "LineString"
            },
            "properties": {
            },
            "type": "Feature"
},
{
            "geometry": {
                "coordinates": [
[116.32776667, 40.07705000],
[116.32783333, 40.07675000],
[116.32786667, 40.07653333],
[116.32793333, 40.07620000],
[116.32798333, 40.0